In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd("../")

'f:\\Files\\DSML\\Condition2Cure'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DriftMonitoringConfig:
    reference_data: Path
    current_data: Path
    output_dir: Path

In [5]:
from Condition2Cure.utils.helpers import *
from Condition2Cure.constants import *
from Condition2Cure.utils.execptions import *

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_drift_monitoring_config(self) -> DriftMonitoringConfig:
        config = self.config.monitoring
        drift_monitoring_config = DriftMonitoringConfig(
            reference_data=Path(config.reference_data),
            current_data=Path(config.current_data),
            output_dir=Path(config.output_dir)
        )
    
        return drift_monitoring_config

In [ ]:
import pandas as pd
from pathlib import Path
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from Condition2Cure.utils.helpers import create_directories
from Condition2Cure import logger


class DriftMonitor:
    def __init__(self, config: DriftMonitoringConfig):
        self.config = config

    def monitoring(self):
        logger.info("Running drift monitoring...")

        ref_path = Path(self.config.reference_data)
        curr_path = Path(self.config.current_data)
        output_dir = Path(self.config.output_dir)

        # Load data
        reference_data = pd.read_csv(ref_path)
        current_data = pd.read_csv(curr_path)

        # Generate Evidently Report
        report = Report(metrics=[DataDriftPreset()])
        report.run(reference_data=reference_data, current_data=current_data)

        # Save reports
        create_directories([output_dir])

        report_path_html = output_dir / "drift_report.html"
        report_path_json = output_dir / "drift_report.json"

        report.save_html(str(report_path_html))
        report.save_json(str(report_path_json))


        logger.info(f"Drift report saved to: {report_path_html}")
        logger.info(f"Drift report (JSON) saved to: {report_path_json}")


In [ ]:
try:
    config = ConfigurationManager()
    drift_monitoring_config = config.get_drift_monitoring_config()
    monitor = DriftMonitor(config=drift_monitoring_config)
    monitor.monitoring()
except Exception as e:
    raise CustomException(str(e), sys)

[2025-06-22 12:12:31,188: INFO: helpers: yaml file: config\config.yaml loaded successfully]
[2025-06-22 12:12:31,188: INFO: helpers: yaml file: config\params.yaml loaded successfully]
[2025-06-22 12:12:31,188: INFO: helpers: yaml file: config\schema.yaml loaded successfully]
[2025-06-22 12:12:31,188: INFO: helpers: created directory at: artifacts]
[2025-06-22 12:12:31,188: INFO: 1951226506: Running drift monitoring...]
[2025-06-22 12:12:39,989: INFO: helpers: created directory at: artifacts\monitoring]
[2025-06-22 12:12:46,288: INFO: 1951226506: Drift report saved to: artifacts\monitoring\drift_report.html]
[2025-06-22 12:12:46,288: INFO: 1951226506: Drift report (JSON) saved to: artifacts\monitoring\drift_report.json]
